In [506]:
# Import libraries
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.orm import Session
from ast import literal_eval

In [507]:
# Read in the CSVs using Pandas
#National Parks
national_parks_df = pd.read_csv("../Resources/national_parks.csv")

#Trails
csv_file = "../Resources/National Park Trail Data.csv"
trail_df = pd.read_csv(csv_file)

#Species
species = "../Resources/National_Parks_Species.csv"
species_df = pd.read_csv(species, keep_default_na=False)

#Visitation
visitors_file = "../Resources/All National Parks Visitation 1904-2016.csv"
visitors_df = pd.read_csv(visitors_file)

#Bear Attacks
bear_attacks_df = pd.read_csv("../Resources/north_america_bear_killings.csv")

# National Parks Transformation

In [508]:
# Read in the national_parks.csv into a dataframe
national_parks_df.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [509]:
# Look at the column names
national_parks_df.columns

Index(['Park Code', 'Park Name', 'State', 'Acres', 'Latitude', 'Longitude'], dtype='object')

In [510]:
# Look at the column data types
national_parks_df.dtypes

Park Code     object
Park Name     object
State         object
Acres          int64
Latitude     float64
Longitude    float64
dtype: object

In [511]:
# Need to rename the columns to match the table columns in nation_parks_db
national_parks_df = national_parks_df.rename(columns={"Park Code":"park_code","Park Name":"park_name",
                                                      "State":"state","Acres":"acres","Latitude":"latitude",
                                                      "Longitude":"longitude"})
national_parks_df.columns

Index(['park_code', 'park_name', 'state', 'acres', 'latitude', 'longitude'], dtype='object')

In [512]:
# Verify no duplicates
national_parks_df.drop_duplicates().shape

(63, 6)

In [513]:
national_parks_df.index.name = "id"
national_parks_df.head()

,park_code,park_name,state,acres,latitude,longitude
id,,,,,,
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


# Trails CSV Transformation

In [514]:
trail_df['_geoloc'] = trail_df['_geoloc'].astype('str').apply(literal_eval)
display(trail_df[trail_df['area_name'] == 'Clayton Co International Park, Jonesboro GA'])
display(trail_df[trail_df['area_name'] == 'Fort Hunt National Park'])
display(trail_df[trail_df['area_name'] == 'Wolf Trap National Park for the Performing Arts'])
trail_df.drop([1219,2475,2618,2629,2524],inplace=True)
trail_df.reset_index(inplace=True,drop=True)

replace_dict = {'Great Sand Dunes National Park and Preserve':'Great Sand Dunes National Park',
 'Congaree National Park Wilderness':'Congaree National Park',
 'Hawaii Volcanoes National Park':"Hawai'i Volcanoes National Park"
 }

trail_df.replace(replace_dict,inplace=True)

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
1219,10259207,Clayton County International Park Trail,"Clayton Co International Park, Jonesboro GA",Jonesboro,Georgia,United States,"{'lat': 33.5312, 'lng': -84.31568}",4.7291,2574.944,36.8808,3,loop,1.0,4.0,26,"['dogs-leash', 'forest', 'kids', 'views', 'wil...","['birding', 'hiking', 'mountain-biking', 'natu...",i


,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
2475,10018025,Mount Vernon Trail,Fort Hunt National Park,Mount Vernon,Virginia,United States,"{'lat': 38.7108, 'lng': -77.08579}",12.0866,29611.856,188.9760,1,out and back,2.0,4.0,118,"['ada', 'dogs-leash', 'kids', 'paved', 'river'...","['birding', 'hiking', 'nature-trips', 'road-bi...",i
2618,10407795,Fort Hunt Park Main Loop,Fort Hunt National Park,Alexandria,Virginia,United States,"{'lat': 38.71518, 'lng': -77.05732}",1.8554,2092.142,9.7536,1,loop,2.0,4.0,4,"['dogs-leash', 'kids', 'paved', 'strollers']","['nature-trips', 'road-biking', 'walking']",i
2629,10407786,Fort Hunt Park Short Loop,Fort Hunt National Park,Alexandria,Virginia,United States,"{'lat': 38.71507, 'lng': -77.05733}",0.8215,1126.538,3.9624,1,loop,2.0,2.0,2,"['dogs-leash', 'kids', 'paved', 'strollers']","['nature-trips', 'road-biking', 'walking']",i


,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
2524,10257350,Wolf Trap Loop Trail,Wolf Trap National Park for the Performing Arts,Vienna,Virginia,United States,"{'lat': 38.93672, 'lng': -77.26292}",7.1119,2896.812,43.8912,1,loop,1.0,4.5,43,"['dogs-leash', 'forest', 'kids', 'river']","['hiking', 'nature-trips', 'walking']",i


In [515]:
trail_df.head()

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18852, 'lng': -149.63156}",24.8931,15610.598,1161.8976,5,out and back,3.0,5.0,423,"['dogs-no', 'forest', 'river', 'views', 'water...","['birding', 'camping', 'hiking', 'nature-trips...",i
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73049, 'lng': -148.91968}",18.0311,6920.162,507.7968,3,out and back,1.0,4.5,260,"['dogs-no', 'forest', 'views', 'wild-flowers',...","['birding', 'camping', 'hiking', 'nature-trips...",i
2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18879, 'lng': -149.631}",17.7821,2896.812,81.9912,1,out and back,3.0,4.5,224,"['dogs-no', 'partially-paved', 'views', 'wildl...","['hiking', 'walking']",i
3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73661, 'lng': -148.915}",16.2674,3379.614,119.7864,1,loop,2.0,4.5,237,"['dogs-no', 'forest', 'lake', 'kids', 'views',...","['birding', 'hiking', 'nature-trips', 'trail-r...",i
4,10236082,Triple Lakes Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73319, 'lng': -148.89682}",12.5935,29772.790,1124.7120,5,out and back,1.0,4.5,110,"['dogs-no', 'lake', 'views', 'wild-flowers', '...","['birding', 'fishing', 'hiking', 'nature-trips...",i


In [516]:
features_df=trail_df[['trail_id','area_name','features']]

In [517]:
features_df['features'] = features_df['features'].astype('str').apply(literal_eval)
features_df = features_df.explode('features')

<ipython-input-517-f3a7cb2bf4fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df['features'] = features_df['features'].astype('str').apply(literal_eval)


In [518]:
features_df.dropna(inplace=True)
features_df.reset_index(inplace=True, drop=True)
features_df.rename(columns={'area_name':'park_name'},inplace=True)

In [519]:
features_df.head()

,trail_id,park_name,features
0,10020048,Kenai Fjords National Park,dogs-no
1,10020048,Kenai Fjords National Park,forest
2,10020048,Kenai Fjords National Park,river
3,10020048,Kenai Fjords National Park,views
4,10020048,Kenai Fjords National Park,waterfall


In [520]:
activities_df=trail_df[['trail_id','area_name','activities']]

In [521]:
activities_df['activities'] = activities_df['activities'].astype('str').apply(literal_eval)
activities_df = activities_df.explode('activities')
activities_df.dropna(inplace=True)
activities_df.reset_index(inplace=True, drop=True)
activities_df.rename(columns={'area_name':'park_name'},inplace=True)
activities_df.head()

<ipython-input-521-6185445fad29>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_df['activities'] = activities_df['activities'].astype('str').apply(literal_eval)


,trail_id,park_name,activities
0,10020048,Kenai Fjords National Park,birding
1,10020048,Kenai Fjords National Park,camping
2,10020048,Kenai Fjords National Park,hiking
3,10020048,Kenai Fjords National Park,nature-trips
4,10020048,Kenai Fjords National Park,trail-running


In [522]:
trail_df_clean = trail_df.drop(columns=['features','activities'])

In [523]:
trail_df_clean.head()

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,units
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18852, 'lng': -149.63156}",24.8931,15610.598,1161.8976,5,out and back,3.0,5.0,423,i
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73049, 'lng': -148.91968}",18.0311,6920.162,507.7968,3,out and back,1.0,4.5,260,i
2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18879, 'lng': -149.631}",17.7821,2896.812,81.9912,1,out and back,3.0,4.5,224,i
3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73661, 'lng': -148.915}",16.2674,3379.614,119.7864,1,loop,2.0,4.5,237,i
4,10236082,Triple Lakes Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73319, 'lng': -148.89682}",12.5935,29772.790,1124.7120,5,out and back,1.0,4.5,110,i


In [524]:
trail_df_clean['latitude'] = ''
trail_df_clean['longitude'] = ''
for i,row in trail_df_clean.iterrows():
    trail_df_clean.iloc[i,16] = trail_df_clean.iloc[i,6]['lat']
    trail_df_clean.iloc[i,17] = trail_df_clean.iloc[i,6]['lng']
    

In [525]:
trail_df_clean.drop(columns='_geoloc',inplace=True)

In [526]:
trail_df_clean.dropna(inplace=True,)

In [527]:
trail_df_clean.reset_index(inplace=True, drop=True)

In [528]:
trail_df_clean.rename(columns={'name':'trail_name','city_name':'city','area_name':'park_name','state_name':'state','country_name':'country'},inplace=True)

In [529]:
trail_df_clean.head()

,trail_id,trail_name,park_name,city,state,country,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,units,latitude,longitude
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,24.8931,15610.598,1161.8976,5,out and back,3.0,5.0,423,i,60.1885,-149.632
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,18.0311,6920.162,507.7968,3,out and back,1.0,4.5,260,i,63.7305,-148.92
2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,17.7821,2896.812,81.9912,1,out and back,3.0,4.5,224,i,60.1888,-149.631
3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,16.2674,3379.614,119.7864,1,loop,2.0,4.5,237,i,63.7366,-148.915
4,10236082,Triple Lakes Trail,Denali National Park,Denali National Park,Alaska,United States,12.5935,29772.790,1124.7120,5,out and back,1.0,4.5,110,i,63.7332,-148.897


In [530]:
activities_df['id'] = activities_df.index + 1
features_df['id'] = features_df.index +1

# Species CSV Transformation

In [531]:
# Printing csv file
species_df.head()

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,,
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,,,
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,,Species of Concern,
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,,,Endangered,
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,,


In [532]:
# Dropping unwanted columns
# Printing the new DataFrame
species_df = species_df.drop(columns=['Order', 'Family', 'Record Status', 'Nativeness', 'Seasonality', 'Conservation Status', 'Unnamed: 13'])
species_df.head()

,Species ID,Park Name,Category,Scientific Name,Common Names,Occurrence,Abundance
0,ACAD-1000,Acadia National Park,Mammal,Alces alces,Moose,Present,Rare
1,ACAD-1001,Acadia National Park,Mammal,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Present,Abundant
2,ACAD-1002,Acadia National Park,Mammal,Canis latrans,"Coyote, Eastern Coyote",Present,Common
3,ACAD-1003,Acadia National Park,Mammal,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Not Confirmed,
4,ACAD-1004,Acadia National Park,Mammal,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Present,Common


In [533]:
# Keeping species that are only "Present" - Keeping rows where value in "Occurrence" column is equal to "Present"
# Printing new DataFrame
species_df = species_df.loc[species_df['Occurrence'] == 'Present']
species_df.head()

,Species ID,Park Name,Category,Scientific Name,Common Names,Occurrence,Abundance
0,ACAD-1000,Acadia National Park,Mammal,Alces alces,Moose,Present,Rare
1,ACAD-1001,Acadia National Park,Mammal,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Present,Abundant
2,ACAD-1002,Acadia National Park,Mammal,Canis latrans,"Coyote, Eastern Coyote",Present,Common
4,ACAD-1004,Acadia National Park,Mammal,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Present,Common
8,ACAD-1008,Acadia National Park,Mammal,Lutra canadensis,"Otter, River Otter",Present,Common


In [534]:
# Renaming columns
species_df = species_df.rename(columns={'Species ID': 'species_id', 'Park Name': 'park_name', 'Category': 'category', 'Scientific Name': 'scientific_name', 'Occurrence': 'occurrence', 'Common Names':'common_names', 'Occurence':'occurence', 'Abundance':'abundance'})
species_df.head()

,species_id,park_name,category,scientific_name,common_names,occurrence,abundance
0,ACAD-1000,Acadia National Park,Mammal,Alces alces,Moose,Present,Rare
1,ACAD-1001,Acadia National Park,Mammal,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Present,Abundant
2,ACAD-1002,Acadia National Park,Mammal,Canis latrans,"Coyote, Eastern Coyote",Present,Common
4,ACAD-1004,Acadia National Park,Mammal,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Present,Common
8,ACAD-1008,Acadia National Park,Mammal,Lutra canadensis,"Otter, River Otter",Present,Common


In [535]:
# Checking for duplicates
# No duplicates
species_df.duplicated()

0         False
1         False
2         False
4         False
8         False
          ...  
119243    False
119244    False
119245    False
119246    False
119247    False
Length: 83278, dtype: bool

In [536]:
# Resetting index
# Printing new DataFrame
species_df = species_df.reset_index(drop=True)
species_df.head()

,species_id,park_name,category,scientific_name,common_names,occurrence,abundance
0,ACAD-1000,Acadia National Park,Mammal,Alces alces,Moose,Present,Rare
1,ACAD-1001,Acadia National Park,Mammal,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Present,Abundant
2,ACAD-1002,Acadia National Park,Mammal,Canis latrans,"Coyote, Eastern Coyote",Present,Common
3,ACAD-1004,Acadia National Park,Mammal,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Present,Common
4,ACAD-1008,Acadia National Park,Mammal,Lutra canadensis,"Otter, River Otter",Present,Common


In [537]:
# Renaming the index to "id"
# Printing the new DataFrame
species_df.index.name = 'id'
species_df.head()

,species_id,park_name,category,scientific_name,common_names,occurrence,abundance
id,,,,,,,
0,ACAD-1000,Acadia National Park,Mammal,Alces alces,Moose,Present,Rare
1,ACAD-1001,Acadia National Park,Mammal,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Present,Abundant
2,ACAD-1002,Acadia National Park,Mammal,Canis latrans,"Coyote, Eastern Coyote",Present,Common
3,ACAD-1004,Acadia National Park,Mammal,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Present,Common
4,ACAD-1008,Acadia National Park,Mammal,Lutra canadensis,"Otter, River Otter",Present,Common


# Parks Visitation CSV Transformation

In [538]:
# Load DF
visitors_df.head()

,Created By,Measure Selector,Year,Date Edit,ScrapeURL,GIS Notes,Gnis Id,Geometry,Metadata,Number of Records,Parkname,Region,State,Unit Code,Unit Name,Unit Type,Visitors,YearRaw
0,Legacy,1,1/1/1904 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1163670,POLYGON,NaN,1,Crater Lake,PW,OR,CRLA,Crater Lake National Park,National Park,1500.0,1904
1,Legacy,1,1/1/1941 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1531834,MULTIPOLYGON,NaN,1,Lake Roosevelt,PW,WA,LARO,Lake Roosevelt National Recreation Area,National Recreation Area,0.0,1941
2,Legacy,1,1/1/1961 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,2055170,MULTIPOLYGON,NaN,1,Lewis and Clark,PW,WA,LEWI,Lewis and Clark National Historical Park,National Historical Park,69000.0,1961
3,Legacy,1,1/1/1935 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1530459,MULTIPOLYGON,NaN,1,Olympic,PW,WA,OLYM,Olympic National Park,National Park,2200.0,1935
4,Legacy,1,1/1/1982 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact denise_kamradt@nps.gov for add...,277263,POLYGON,NaN,1,Santa Monica Mountains,PW,CA,SAMO,Santa Monica Mountains National Recreation Area,National Recreation Area,468144.0,1982


In [539]:
# Grab only National Parks from the DataFrame
visitors_df = visitors_df.loc[visitors_df["Unit Type"] == "National Park"]

# filter out rows that do not contain a numeric year
visitors_df = visitors_df.loc[visitors_df["YearRaw"] != "Total"]

# Create filtered dataframe with specific columns
visitors_cols = ["Region", "State", "Unit Name", "Visitors", "YearRaw"]
visitors_transformed= visitors_df[visitors_cols].copy()



# Rename the column headers for clarification
visitors_transformed = visitors_transformed.rename(columns={"Region":"region",
                                                            "State":"state",
                                                            "Unit Name": "park_name",
                                                            "Visitors":"visitors",
                                                            "YearRaw": "year"})
# Reset the index
visitors_transformed.reset_index(drop=True, inplace=True)

# Create the ID column based off of the index
visitors_transformed['id'] = visitors_transformed.index + 1

# Set the index to the ID
visitors_transformed.set_index('id',drop=True, inplace=True)

visitors_transformed.head()

,region,state,park_name,visitors,year
id,,,,,
1,PW,OR,Crater Lake National Park,1500.0,1904
2,PW,WA,Olympic National Park,2200.0,1935
3,NE,ME,Acadia National Park,64000.0,1919
4,IM,TX,Big Bend National Park,1409.0,1944
5,SE,FL,Everglades National Park,7482.0,1948


# Bear Attacks CSV Transformation

In [540]:
#inspect the columns
bear_attacks_df.columns

Index(['Name', ' age', 'gender', 'Date', 'Month', 'Year', 'Type', 'Location',
       'Description', 'Type of bear', 'Hunter', 'Grizzly', 'Hikers',
       'Only one killed'],
      dtype='object')

In [541]:
#age column has a leading space - remove that and make all others lower case
bear_attacks_df = bear_attacks_df.rename(columns={'Name': 'name', ' age': 'age', 'Date': 'date', 'Type': 'type', 'Location': 'location', 'Description': 'description', 'Type of bear': 'bear_type'})


In [542]:
#we only care about attacks in National Parks - filter for those - these will include Canada's National Parks as well as USA National Parks
in_parks = bear_attacks_df['location'].str.contains('National Park', case=True, regex=False)

#select the relevant columns
in_parks_df = bear_attacks_df[in_parks]
in_parks_df = in_parks_df[['name', 'age', 'date', 'type', 'location', 'description', 'bear_type']]


In [543]:
#check for any duplicate records
in_parks_df.shape

(32, 7)

In [544]:
#there are no duplicate records
in_parks_df.drop_duplicates().shape


(32, 7)

In [545]:
#reset the index - this will be the sql primary key
in_parks_df = in_parks_df.reset_index(drop=True)
#give index column name 'id'
in_parks_df.index.name = 'id'

In [546]:
in_parks_df

,name,age,date,type,location,description,bear_type
id,,,,,,,
0,Lance Crosby,63,2015-08-07,Wild,"Yellowstone National Park, Wyoming","Crosby, an employee at a medical clinic in the...",Brown bear
1,Richard White,49,2012-08-24,Wild,"Denali National Park, Alaska",White was backpacking alone along the Toklat R...,Brown bear
2,John Wallace,59,2011-08-24,Wild,"Yellowstone National Park, Wyoming",Wallace's remains were found by hikers on the ...,Brown bear
3,Brian Matayoshi,57,2011-07-06,Wild,"Yellowstone National Park, Wyoming",Matayoshi and his wife were hiking the Wapiti ...,Brown bear
4,Glenda Ann Bradley,50,2000-05-21,Wild,"Great Smoky Mountains National Park, Tennessee",Bradley was attacked and partially consumed by...,Black bear
5,Timothy Treadwell,46,2003-10-05,Wild,"Katmai National Park, Alaska",Treadwell and Huguenard's corpses were found b...,Brown bear
6,Amie Huguenard,37,2003-10-05,Wild,"Katmai National Park, Alaska",Treadwell and Huguenard's corpses were found b...,Brown bear
7,Craig Dahl,26,1998-05-17,Wild,"Glacier National Park, Montana",Dahl's partially consumed remains were found t...,Brown bear
8,Robert Bell,33,1996-08-23,Wild,"Gates of the Arctic National Park, Alaska",Bell was killed while hiking with a friend nea...,Brown bear


In [547]:
in_parks_df['location'] = in_parks_df['location'].str.split(',').str[0]
in_parks_df

,name,age,date,type,location,description,bear_type
id,,,,,,,
0,Lance Crosby,63,2015-08-07,Wild,Yellowstone National Park,"Crosby, an employee at a medical clinic in the...",Brown bear
1,Richard White,49,2012-08-24,Wild,Denali National Park,White was backpacking alone along the Toklat R...,Brown bear
2,John Wallace,59,2011-08-24,Wild,Yellowstone National Park,Wallace's remains were found by hikers on the ...,Brown bear
3,Brian Matayoshi,57,2011-07-06,Wild,Yellowstone National Park,Matayoshi and his wife were hiking the Wapiti ...,Brown bear
4,Glenda Ann Bradley,50,2000-05-21,Wild,Great Smoky Mountains National Park,Bradley was attacked and partially consumed by...,Black bear
5,Timothy Treadwell,46,2003-10-05,Wild,Katmai National Park,Treadwell and Huguenard's corpses were found b...,Brown bear
6,Amie Huguenard,37,2003-10-05,Wild,Katmai National Park,Treadwell and Huguenard's corpses were found b...,Brown bear
7,Craig Dahl,26,1998-05-17,Wild,Glacier National Park,Dahl's partially consumed remains were found t...,Brown bear
8,Robert Bell,33,1996-08-23,Wild,Gates of the Arctic National Park,Bell was killed while hiking with a friend nea...,Brown bear


# Data Loads to Postgres

In [548]:
# Open database connection
db_conn = "postgres:postgres@localhost:5432/national_parks_db"
engine = create_engine(f"postgresql://{db_conn}")

# Check the table names for accuracy
inspector = inspect(engine)
inspector.get_table_names()

[]

In [549]:
# Load the National Parks dataframe into the postgres table
national_parks_df.to_sql(name="national_parks", con=engine, if_exists='append', index=True)

# Verify the data was loaded in the table
pd.read_sql_query("select * from national_parks", con=engine).head()


,id,park_code,park_name,state,acres,latitude,longitude
0,0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [550]:
# Load the Trails database tables
trail_df_clean.to_sql(name='park_trails', con=engine, if_exists='append', index=False)
activities_df.to_sql(name='trail_activities', con=engine, if_exists='append', index=False)
features_df.to_sql(name='trail_features', con=engine, if_exists='append', index=False)

# Verify the data was loaded in the table
pd.read_sql_query("select * from park_trails", con=engine).head()

,trail_id,trail_name,park_name,city,state,country,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,units,latitude,longitude
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,24.8931,15610.598,1161.8976,5,out and back,3.0,5.0,423,i,60.18852,-149.63156
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,18.0311,6920.162,507.7968,3,out and back,1.0,4.5,260,i,63.73049,-148.91968
2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,17.7821,2896.812,81.9912,1,out and back,3.0,4.5,224,i,60.18879,-149.63100
3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,16.2674,3379.614,119.7864,1,loop,2.0,4.5,237,i,63.73661,-148.91500
4,10236082,Triple Lakes Trail,Denali National Park,Denali National Park,Alaska,United States,12.5935,29772.790,1124.7120,5,out and back,1.0,4.5,110,i,63.73319,-148.89682


In [551]:
# Loading Species DataFrame into database
species_df.to_sql(name='species', con=engine, if_exists='append', index=True)

# Verify the data was loaded in the table
pd.read_sql_query("select * from species", con=engine).head()

,id,species_id,park_name,category,scientific_name,common_names,occurrence,abundance
0,0,ACAD-1000,Acadia National Park,Mammal,Alces alces,Moose,Present,Rare
1,1,ACAD-1001,Acadia National Park,Mammal,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Present,Abundant
2,2,ACAD-1002,Acadia National Park,Mammal,Canis latrans,"Coyote, Eastern Coyote",Present,Common
3,3,ACAD-1004,Acadia National Park,Mammal,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Present,Common
4,4,ACAD-1008,Acadia National Park,Mammal,Lutra canadensis,"Otter, River Otter",Present,Common


In [552]:
# Load Visitors dataframe into the database
visitors_transformed.to_sql(name='parks_visitation', con=engine, if_exists='append', index=True)

# Verify the data was loaded in the table
pd.read_sql_query("select * from parks_visitation", con=engine).head()

,id,region,state,park_name,visitors,year
0,1,PW,OR,Crater Lake National Park,1500.0,1904
1,2,PW,WA,Olympic National Park,2200.0,1935
2,3,NE,ME,Acadia National Park,64000.0,1919
3,4,IM,TX,Big Bend National Park,1409.0,1944
4,5,SE,FL,Everglades National Park,7482.0,1948


In [553]:
#write Bear Attack data to sql
in_parks_df.to_sql(name='bear_attacks', con=engine, if_exists='append', index=True)

# Verify the data was loaded in the table
pd.read_sql_query("select * from bear_attacks", con=engine).head()

,id,name,age,date,type,location,description,bear_type
0,0,Lance Crosby,63,2015-08-07,Wild,Yellowstone National Park,"Crosby, an employee at a medical clinic in the...",Brown bear
1,1,Richard White,49,2012-08-24,Wild,Denali National Park,White was backpacking alone along the Toklat R...,Brown bear
2,2,John Wallace,59,2011-08-24,Wild,Yellowstone National Park,Wallace's remains were found by hikers on the ...,Brown bear
3,3,Brian Matayoshi,57,2011-07-06,Wild,Yellowstone National Park,Matayoshi and his wife were hiking the Wapiti ...,Brown bear
4,4,Glenda Ann Bradley,50,2000-05-21,Wild,Great Smoky Mountains National Park,Bradley was attacked and partially consumed by...,Black bear
